In [1]:
import scipy as sp 
import numpy as np 
import pandas as pd
from FileExporter import create_download_link

In [18]:
A = pd.read_csv('./msresist/data/20180817_JG_AM_TMT10plex_R1_psms.csv', header=0)
B = pd.read_csv('./msresist/data/20190214_JG_AM_PC9_AXL_TMT10_AC28_R2_PSMs.csv', header=0)
C = pd.read_csv('./msresist/data/CombinedBR3_TR1&2.csv', header=0)

#Data
a, b, c = A.iloc[:, 2:12], B.iloc[:, 2:12], C.iloc[:, 2:12]
t = list(a.columns)

#Peptide Sequences
Aseqs, Bseqs, Cseqs = A.iloc[:, 0], B.iloc[:, 0], C.iloc[:, 0]

#Protein Names 
Anames, Bnames, Cnames = [], [], []
x = list(map(lambda  v : Anames.append(v.split("OS")[0]), A.iloc[:,1]))
y = list(map(lambda  v : Bnames.append(v.split("OS")[0]), B.iloc[:,1]))
z = list(map(lambda  v : Cnames.append(v.split("OS")[0]), C.iloc[:,1]))

A['Master Protein Descriptions'] = Anames
B['Master Protein Descriptions'] = Bnames
C['Master Protein Descriptions'] = Cnames

There are overlapping peptides across all 3 BRs... eg: gSTAENAEyLR-1, qNVPVINITyDSTPEDVk, gSHQISLDNPDyQQDFFPk, ... Same proteins have different descriptions which it's why I'm renaming them to .split("OS")[0]. 

In [3]:
# A[A["Master Protein Descriptions"].str.contains("Epidermal growth factor")]

In [4]:
# B[B["Master Protein Descriptions"].str.contains("Epidermal growth factor")]

In [5]:
# C[C["Master Protein Descriptions"].str.contains("Epidermal growth factor")]

In [14]:
#Concatenate 3 BR
frames = [A,B,C]
ABC = pd.concat(frames)
print("shape of concatenated matrix:", ABC.shape)

#count number of duplicates across data sets
dups = pd.pivot_table(ABC, index=['Master Protein Descriptions', 'peptide-phosphosite'], aggfunc = "size")
my_dict = {i:list(dups).count(i) for i in list(dups)}
print("total number of recurrences:", my_dict)

#compute mean across duplicates 
func = {}
for i in t:
    func[i] = np.mean
ABC_avg = pd.pivot_table(ABC, values = t, index = ['Master Protein Descriptions', 'peptide-phosphosite'], aggfunc = func)

dups2 = ABC_avg.pivot_table(index=['Master Protein Descriptions', 'peptide-phosphosite'], aggfunc = "size")
print("shape of averaged matrix:", ABC_avg.shape)
create_download_link(pd.DataFrame(dups), "RecPeptidesAcrossBRs")

shape of concatenated matrix: (1496, 12)
total number of recurrences: {2: 246, 1: 620, 3: 128}
shape of averaged matrix: (994, 10)


In [7]:
create_download_link(pd.DataFrame(ABC_avg), "Merged_BRs")

In [13]:
create_download_link(pd.DataFrame(ABC), "Conc_BRs")